# Calculate Text-Similarities by LDA Model
In this step, we hope to use the available data to train a pretty good LDA model, and then we the model to make a vector presentation for every summary of papers. we could ,therefore, use the vectors to calculate score of similarities.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/My Drive/Research/Data/"
path = path+'My_result/Mallet/mallet-2.0.8/bin/mallet' 

os.chdir(path)
os.listdir(path)

Mounted at /content/drive/


['english.txt',
 '2019_2020_all_data_final.csv',
 '2014_abs',
 '2015_abs',
 '2016_abs',
 '2017_abs',
 '2018_abs',
 'My_Process',
 'all_data_statistic.csv',
 'top_category_details.csv',
 'top5_statistic_months.csv',
 'top5_statistics_col_1.csv',
 'cs_summary',
 'My_result']

In [ ]:
path="/content/drive/My Drive/Research/Data/My_result/Mallet/mallet/bin/"

os.chdir(path)
os.listdir(path)

['vectors2vectors',
 'vectors2topics',
 'vectors2classify',
 'mallet',
 'mallethon',
 'prepend-license.sh',
 'vectors2info',
 'text2classify',
 'text2vectors',
 'mallet.bat',
 'classifier2info',
 'csv2vectors',
 'svmlight2vectors',
 'csv2classify']

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
import nltk

In [ ]:
from gensim.test.utils import datapath

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import csv

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

# spacy for lemmatization
import spacy
import en_core_web_lg
#nlp = spacy.load('en_core_web_lg')
nlp = en_core_web_lg.load()

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
#import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

LookupError: ignored

In [ ]:
# 加载数据的函数
def read_summary(file_path):
  text = pd.read_csv(file_path,usecols=[3],encoding='ISO-8859-1',header=None,names=['summary'])

  # pd --> list
  data_list = []
  for index in range(1,len(text['summary'])):
     row_info = text.loc[index]
     summary = row_info[0]
     data_list.append(summary)
  return data_list

In [ ]:
text_2014 = read_summary(path+'My_Process/Summary_2014.csv')
text_2015 = read_summary(path+'My_Process/Summary_2015.csv')
text_2016 = read_summary(path+'My_Process/Summary_2016.csv')
text_2017 = read_summary(path+'My_Process/Summary_2017.csv')
text_2018 = read_summary(path+'My_Process/Summary_2018.csv')
text_2019 = read_summary(path+'My_Process/Summary_2019.csv')
text_2020 = read_summary(path+'My_Process/Summary_2020.csv')

text_all =[]
text_all.extend(text_2014)
text_all.extend(text_2015)
text_all.extend(text_2016)
text_all.extend(text_2017)
text_all.extend(text_2018)
text_all.extend(text_2019)
text_all.extend(text_2020)

print(len(text_all))

165165


In [ ]:
#加载index_year  （包含data里的数据划分节点数）
index_year_ = np.load(path+'My_Process/index_year.npy')
index_year_=index_year_.tolist()
index_year_

In [ ]:
#移除标点符号
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def cleandata(text):
  # Remove new line characters
  text_result = [re.sub('\s+', ' ', sent) for sent in text]

  # Remove distracting single quotes
  text_result = [re.sub("\'", "", sent) for sent in text_result]

  text_result = list(sent_to_words(text_result))

  return text_result


#保存二维列表到文件
def save_list(list1,file_path):
    file2 = open(file_path, 'w')
    for i in range(len(list1)):
        for j in range(len(list1[i])):
            file2.write(str(list1[i][j]))              # write函数不能写int类型的参数，所以使用str()转化
            file2.write('\t')                          # 相当于Tab一下，换一个单元格
        file2.write('\n')                              # 写完一行立马换行
    file2.close()


# 读文件里面的数据转化为二维列表
def read_list(file_path):
    file1 = open(file_path, "r")
    list_row =file1.readlines()
    list_source = []
    for i in range(len(list_row)):
        column_list = list_row[i].strip().split("\t")  # 每一行split后是一个列表
        list_source.append(column_list)                # 在末尾追加到list_source
    for i in range(len(list_source)):  # 行数
        for j in range(len(list_source[i])):  # 列数
            list_source[i][j]=list_source[i][j]
    file1.close()
    return list_source

In [ ]:
data_words = cleandata(text_all)

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words and len(word) >= 2] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags):
    texts_out = []
    number = 0
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])#只要这四种词性的词

        number = number + 1
        if number % 200==0:
          print('we have processed ',number,' text')

    return texts_out


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['for', 'learning', 'problem', 'of', 'radial_basis_function', 'process', 'neural', 'network', 'rbf', 'pnn', 'an', 'optimization', 'training', 'method', 'based', 'on', 'ga', 'combined', 'with', 'sa', 'is', 'proposed', 'in', 'this', 'paper', 'through', 'building', 'generalized', 'fra_chet_distance', 'to', 'measure', 'similarity', 'between', 'time', 'varying', 'function', 'samples', 'the', 'learning', 'problem', 'of', 'radial_basis', 'centre', 'functions', 'and', 'connection', 'weights', 'is', 'converted_into', 'the', 'training', 'on', 'corresponding', 'discrete', 'sequence', 'coefficients', 'network', 'training', 'objective', 'function', 'is', 'constructed', 'according', 'to', 'the', 'least_square', 'error', 'criterion', 'and', 'global', 'optimization', 'solving', 'of', 'network', 'parameters', 'is', 'implemented', 'in', 'feasible', 'solution', 'space', 'by', 'use', 'of', 'global', 'optimization', 'feature', 'of', 'ga', 'and', 'probabilistic', 'jumping', 'property', 'of', 'sa', 'the', 'e

In [ ]:
#data_words

In [ ]:
## 正式制作词袋
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'VERB'])

print(data_lemmatized[:1])

we have processed  200  text
we have processed  400  text
we have processed  600  text
we have processed  800  text
we have processed  1000  text
we have processed  1200  text
we have processed  1400  text
we have processed  1600  text
we have processed  1800  text
we have processed  2000  text
we have processed  2200  text
we have processed  2400  text
we have processed  2600  text
we have processed  2800  text
we have processed  3000  text
we have processed  3200  text
we have processed  3400  text
we have processed  3600  text
we have processed  3800  text
we have processed  4000  text
we have processed  4200  text
we have processed  4400  text
we have processed  4600  text
we have processed  4800  text
we have processed  5000  text
we have processed  5200  text
we have processed  5400  text
we have processed  5600  text
we have processed  5800  text
we have processed  6000  text
we have processed  6200  text
we have processed  6400  text
we have processed  6600  text
we have proces

In [ ]:
save_list(data_lemmatized,path+'My_Process/Lemmatize_data_14to20.txt')

In [ ]:
data_lemmatized = read_list(path+'My_Process/Lemmatize_data_14to20.txt')

In [ ]:
## 训练LDA模型
data = data_lemmatized

corpus_dictionary = Dictionary(data)
corpus = [corpus_dictionary.doc2bow(text) for text in data]


In [ ]:
## 使用 gensim封装的的LDA模型 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=corpus_dictionary,
                                           num_topics=40, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50000,
                                           passes=60,
                                           alpha='auto',
                                           per_word_topics=True)
#pprint(lda_model.print_topics())

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=corpus_dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.4935443811188227


In [ ]:
## 优化，使用java封装的LDA模型
mallet_path = path+'mallet'  # update this path
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=30,id2word=corpus_dictionary )

In [ ]:
pprint(lda_model.print_topics())

[(3,
  '0.058*"flow" + 0.057*"vehicle" + 0.051*"traffic" + 0.032*"location" + '
  '0.019*"road" + 0.016*"time" + 0.015*"route" + 0.015*"drive" + '
  '0.014*"mobility" + 0.014*"safety"'),
 (25,
  '0.062*"protocol" + 0.035*"distribute" + 0.033*"communication" + '
  '0.024*"blockchain" + 0.018*"transaction" + 0.018*"client" + 0.016*"server" '
  '+ 0.015*"consensus" + 0.015*"system" + 0.013*"use"'),
 (4,
  '0.028*"cost" + 0.028*"strategy" + 0.026*"mechanism" + 0.021*"utility" + '
  '0.015*"value" + 0.015*"show" + 0.014*"game" + 0.012*"fairness" + '
  '0.012*"market" + 0.011*"price"'),
 (20,
  '0.092*"match" + 0.086*"rank" + 0.053*"track" + 0.044*"measurement" + '
  '0.041*"alignment" + 0.040*"tracking" + 0.037*"matching" + 0.034*"recovery" '
  '+ 0.025*"registration" + 0.020*"template"'),
 (27,
  '0.050*"attack" + 0.038*"security" + 0.024*"privacy" + 0.017*"system" + '
  '0.015*"use" + 0.014*"datum" + 0.012*"base" + 0.011*"propose" + '
  '0.011*"device" + 0.010*"information"'),
 (14,
  '0.

In [ ]:
temp_file = datapath(path+'My_result/LDA_t30_p50_c50000')
lda_model.save(temp_file)

In [ ]:
corpus[1]

In [ ]:
lda_model.get_document_topics(corpus[0])

[(5, 0.023800734),
 (6, 0.017512003),
 (7, 0.13979049),
 (10, 0.03721434),
 (12, 0.6854655),
 (13, 0.03574851),
 (14, 0.014548144)]

In [ ]:
list1 = ['aaa','ss','asd']
list2 = ['sss','bb','fed']

list1.extend(list2)
list1

['aaa', 'ss', 'asd', 'sss', 'bb', 'fed']

In [ ]:
list1.count()

TypeError: ignored